# Annotation CB LLM(e.g on ledgar dataset)
---
Goal of the notebook: .

Inputs of the notebook:
- Dataset as pandas dataframe : with a column "text" and a "label" column
- Concepts list in a pickle file 

Output of the notebook:
- dataframe and dataloader of the augmented dataset with concepts


In [1]:
# !pip install -r requirements.txt

In [3]:
import sys
sys.path.append('../../run_experiments/')
sys.path.append('../../run_experiments/scripts')
sys.path.append('../../run_experiments/models')
sys.path.append('../../run_experiments/data')

import gc
import torch

torch.cuda.empty_cache()
gc.collect()

0

In [13]:
#code for autoreload script associated with jupyter notebook
%load_ext autoreload
%autoreload 2

# 1. Data Loading and Preparation


In [3]:
# import config
from config_ledgar import Config
config = Config()

In [4]:
from data_ledgar import prepare_ledgar_data
train_loader, test_loader, val_loader, train_df, val_df, test_df = prepare_ledgar_data(config)

Chargement des fichiers sauvegardés...


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

# 2. retrieve concepts list

ledgar

In [10]:
Amendments_concepts = ['Modification rights', 'Amendment procedures', 'Notice requirements', 'Approval mechanisms',
                       'Integration with original agreement', 'Format requirements', 'Severability of amendments', 'Retroactive application',
                       'Waiver limitations', 'Amendment thresholds', 'Unilateral amendment rights', 'Amendment restrictions', 
                       'Prior versions validity', 'Amendment documentation',
                       'Version control mechanisms', 'Material change provisions']

Terminations_concepts = ['Termination rights', 'Notice periods', 'Termination for cause', 'Termination for convenience',
                         'Cure periods', 'Effect of termination', 'Wind-down procedures', 'Return of property/information',
                         'Early termination penalties', 'Mutual termination provisions', 'Partial termination rights',
                         'Force majeure termination', 'Change of control provisions', 'Bankruptcy/insolvency triggers',
                         'Performance-based termination', 'Regulatory/legal change termination', 'Exit assistance',
                         'Termination certification requirements', 'Post-termination restrictions', 'Transition obligations']

Survival_concepts = ['Post-termination obligations', 'Duration of surviving terms', 'Identification of specific clauses',
                     'Confidentiality persistence', 'IP rights retention', 'Indemnification continuation',
                     'Limitation of liability', 'Dispute resolution mechanisms', 'Payment obligations survival',
                     'Non-compete/non-solicitation persistence', 'Representations/warranties survival',
                     'Record-keeping requirements', 'Audit rights continuation', 'Escrow arrangements', 
                     'Insurance obligations', 'Remedies availability post-termination', 'Data protection obligations',
                     'Perpetual rights', 'Transitional licenses', 'Legal compliance requirements']

Terms_concepts = ['Duration specifications', 'Commencement date', 'Expiration conditions', 'Renewal mechanisms',
                  'Term length', 'Condition precedents', 'Milestone-based periods', 'Extension options',
                  'Initial term vs. renewal term distinctions', 'Notice requirements for non-renewal',
                  'Evergreen provisions', 'Term modification triggers', 'Retroactive effective dates',
                  'Trial/probation periods', 'Minimum commitment periods', 'Maximum term limitations',
                  'Regulatory term constraints', 'Performance-based extensions', 'Phase-in schedules', 
                  'Renegotiation periods', 'Term acceleration provisions', 'Rolling term provisions']

concept_ledgar = Amendments_concepts + Survival_concepts + Terms_concepts + Terminations_concepts

concept_data_legdar_by_class = {
                                'Amendments': Amendments_concepts, 
                                'Survival': Survival_concepts, 
                                'Terms': Terms_concepts, 
                                'Terminations': Terminations_concepts
                               }
data = [{'ledgar':concept_data_legdar_by_class}]

movies, agnews, medical, dbpedia

In [ ]:
# import pickle 
# # open a file, where you stored the pickled data
# file = open('concepts_CB_LLM.pkl', 'rb')

# # dump information to that file
# data = pickle.load(file)

# # close the file
# file.close()
# data[0].keys()

# 3. compute cosine SCORE

In [6]:
from annotation_CB_LLM import get_annotation_CB_LLM

In [ ]:
for i,c in enumerate(concept_ledgar):
    similarities = get_annotation_CB_LLM(train_df.text.tolist(), c, model = "all-mpnet-base-v2")
    train_df[c] = similarities.tolist()[0]


In [ ]:
train_df.to_csv(f'{config.SAVE_PATH_CONCEPTS}/df_with_topics_v4_CB_LLM.csv')

In [ ]:
for i,c in enumerate(concept_ledgar):
    similarities = get_annotation_CB_LLM(test_df.text.tolist(), c, model)
    test_df[c] = similarities.tolist()[0]


In [ ]:
test_df.to_csv(f'{config.SAVE_PATH_CONCEPTS}/df_with_topics_v4_CB_LLM_test.csv')

## ACC filtering

In [4]:
from annotation_CB_LLM import mapping_concept_label

In [ ]:
# ledgar has 100 inital label
caption = ["Adjustments", "Agreements", "Amendments", "Anti-Corruption Laws", "Applicable Laws", "Approvals", "Arbitration", 
           "Assignments", "Assigns", "Authority", "Authorizations", "Base Salary", "Benefits", "Binding Effects", "Books", "Brokers",
           "Capitalization", "Change In Control", "Closings", "Compliance With Laws", "Confidentiality", "Consent To Jurisdiction", 
           "Consents", "Construction", "Cooperation", "Costs", "Counterparts", "Death", "Defined Terms", "Definitions", "Disability", 
           "Disclosures", "Duties", "Effective Dates", "Effectiveness", "Employment", "Enforceability", "Enforcements", "Entire Agreements", 
           "Erisa", "Existence", "Expenses", "Fees", "Financial Statements", "Forfeitures", "Further Assurances", "General", "Governing Laws", 
           "Headings", "Indemnifications", "Indemnity", "Insurances", "Integration", "Intellectual Property", "Interests", "Interpretations", 
           "Jurisdictions", "Liens", "Litigations", "Miscellaneous", "Modifications", "No Conflicts", "No Defaults", "No Waivers", 
           "Non-Disparagement", "Notices", "Organizations", "Participations", "Payments", "Positions", "Powers", "Publicity",
           "Qualifications", "Records", "Releases", "Remedies", "Representations", "Sales", "Sanctions", "Severability", "Solvency",
           "Specific Performance", "Submission To Jurisdiction", "Subsidiaries", "Successors", "Survival", "Tax Withholdings", "Taxes",
           "Terminations", "Terms", "Titles", "Transactions With Affiliates", 
           "Use Of Proceeds", "Vacations", "Venues", "Vesting", "Waiver Of Jury Trials", "Waivers", "Warranties", "Withholdings"]

dict = { pos : name for pos,name in enumerate(caption)}

In [ ]:
import os
import json
with open(os.path.join(config.SAVE_PATH_CONCEPTS, "label_mapping.json"), 'r') as f:
    labelling_json = json.load(f)
labelling_json
# with open(file_path, 'r') as f:
#     label_to_id = json.load(f)

[dict[int(a)] for a in labelling_json.keys()]



In [ ]:
labelling_json

In [ ]:
# dictionary_ledgar = {0: 'documentary', 1: 'comedy', 2: 'horror', 3: 'western'}
# {'Amendments': 0, 'Survival': 1, 'Terminations': 2, 'Terms': 3}
dictionary_ledgar = {0: 'Amendments', 1: 'Survival', 2: 'Terminations', 3: 'Terms'}



In [ ]:
train_df['label'] = train_df['label'].map(dictionary_ledgar)
train_df = train_df.reset_index(drop=True)
train_df

In [ ]:
for i in range(train_df.shape[0]):
    print(i)
    label = train_df['label'][i].values
    for c in concept_all:
        mapping = mapping_concept_label('ledgar', label, c, data)
        if mapping == 0 :
            train_df[c][i]=0
        else:
            pass

In [ ]:
# Dictionnaire initial
dictionary_ledgar = {0: 'Amendments', 1: 'Survival', 2: 'Terminations', 3: 'Terms'}

# Inverser le dictionnaire
label_to_int = {v: k for k, v in dictionary_ledgar.items()}

# Appliquer le mapping inverse
train_df['label'] = train_df['label'].map(label_to_int)

train_df.to_csv(config.SAVE_PATH_CONCEPTS + '/df_with_topics_v4_CB_LLM_ACC.csv')


# test 

In [ ]:
test_df['label'] = test_df['label'].map(dictionary_ledgar)
test_df = test_df.reset_index(drop=True)
test_df

In [ ]:
for i in range(test_df.shape[0]):
    print(i)
    label = test_df['label'][i]
    for c in concept_all:
        mapping = mapping_concept_label('ledgar', label, c, data)
        if mapping == 0 :
            test_df[c][i]=0
        else:
            pass

In [ ]:
# Dictionnaire initial
dictionary_ledgar = {0: 'Amendments', 1: 'Survival', 2: 'Terminations', 3: 'Terms'}

# Inverser le dictionnaire
label_to_int = {v: k for k, v in dictionary_ledgar.items()}

# Appliquer le mapping inverse
test_df['label'] = test_df['label'].map(label_to_int)

test_df.to_csv(config.SAVE_PATH_CONCEPTS + '/df_with_topics_v4_CB_LLM_ACC_test.csv')
